In [1]:
# labeled_data = "./test_small_withlabels.txt"
# result_data = "./kaggle-results.txt"
# error_analysis = "./error_analysis.txt"
# dep_tree = "./test-tweebo.processed.txt"

labeled_data = "./test-cv-labeled3.txt"
result_data = "./kaggle-results3.txt"
error_analysis = "./error_analysis_cv.txt"
dep_tree = "./train-tweebo.processed.txt"



In [ ]:
import sys
import argparse

In [27]:
def get_dependency_data(labeled_data, result_data, dep_tree): #Get dependency data
    with open(labeled_data) as t:
        input_lines = t.readlines()
        labeled_temp = []
        lbl = []
        arr = []
        arr2 = []
        for line in input_lines:
            try:
                if line in ['\n', '\r\n']:
                    sentence = ' '.join(str(x) for x in arr) 
                    labeled_temp.append(sentence.strip())
                    lbl.append(arr2)
                    arr = []
                    arr2 = []
                else:
                    (word, tag) = line.strip().split()
                    arr.append(word)
                    arr2.append((word, tag))
            except ValueError:
                continue

    with open(result_data) as t:
        input_lines = t.readlines()
        result_temp = []
        arr = []
        for line in input_lines:
            try:
                if line in ['\n', '\r\n']:
                    result_temp.append(arr)
                    arr = []
                else:
                    tag = line.strip()
                    arr.append(tag)
            except ValueError:
                continue

    with open(dep_tree) as t:
        input_lines = t.readlines()
        tree_temp = []
        tree = []
        arr = []
        arr2 = []
        for line in input_lines:
            try:
                if line in ['\n', '\r\n']:
                    sentence = ' '.join(str(x) for x in arr) 
                    tree_temp.append(sentence.strip())
                    tree.append(arr2)
                    arr = []
                    arr2 = []
                else:
                    node = line.strip().split()
                    arr.append(node[1])
                    arr2.append(node)
            except ValueError:
                continue

    labeled_indexes = []

    for line in labeled_temp:
        ind = tree_temp.index(line)
        labeled_indexes.append(ind)
        
    return lbl, labeled_indexes, tree, result_temp
                

In [37]:
def get_error_analysis(lbl, labeled_indexes, tree, result_temp):
    tweets = []
    arr = []
    ind = 0
    match = 1
    for t in range(len(lbl)):
        node = tree[labeled_indexes[ind]]
        for t1, t2, t3 in zip(lbl[t], result_temp[t], node):
            (word, tag) = t1
            guess = t2
            (w_id, word, tag2, pos_tag, head, rel) = t3
            if tag != guess.strip():
                match = 0
            arr.append((w_id, word, pos_tag, head, rel, tag, guess.strip()))
        ind += 1

        if not match:
            tweets.append(arr)
        match = 1
        arr = []

    return tweets

[('1', 'Soulja', '^', '2', 'MWE', 'B', 'B'), ('2', 'Boy', '^', '4', '_', 'I', 'I'), ('3', '|', 'G', '4', '_', 'O', 'O'), ('4', 'TheDeAndreWay', '^', '0', '_', 'O', 'O'), ('5', '.', ',', '-1', '_', 'O', 'O'), ('6', 'com', 'N', '0', '_', 'O', 'O'), ('7', '-', ',', '-1', '_', 'O', 'O'), ('8', 'The', 'D', '10', '_', 'O', 'B'), ('9', 'DeAndre', '^', '10', 'MWE', 'O', 'I'), ('10', 'Way', '^', '11', '_', 'O', 'I'), ('11', 'in', 'P', '0', '_', 'O', 'O'), ('12', 'stores', 'N', '11', '_', 'O', 'O'), ('13', 'Nov', '^', '12', '_', 'O', 'O'), ('14', '2', '$', '13', 'MWE', 'O', 'O'), ('15', ',', ',', '-1', '_', 'O', 'O'), ('16', '2010', '$', '0', '_', 'O', 'O'), ('17', 'souljaboytellem-iga.ning.com', 'U', '-1', '_', 'O', 'O'), ('18', 'via', 'P', '16', '_', 'O', 'O'), ('19', '@SouljaBoy', '@', '-1', '_', 'O', 'O'), ('20', '!', ',', '-1', '_', 'O', 'O'), ('21', '#Blessed', '#', '-1', '_', 'O', 'O')]


In [38]:
def write_error_analysis_file(file_name, tweets):
    open(file_name, 'w').close()
    with open (file_name, 'w') as f: 
        for tweet in tweets:
            for word in tweet:
                f.write(word[0] + "\t" + word[1] + "\t\t" + word[2] + "\t" +word[3] + "\t" + word[4] + "\t" + word[5] + "\t" + word[6] +"\n")
            f.write("\n")

In [ ]:
def main(argv):    
    parser = argparse.ArgumentParser()
    parser.add_argument('-labeled-data', help='Path to labeled data', dest='lbl', required=True)
    parser.add_argument('-guessed-data', help='Path to guessed tags', dest='guess', required=True)
    parser.add_argument('-dep-tree-data', help='Path fo dependency tree file', dest='dep', required=True)
    parser.add_argument('-output-file', help='Path to error analysis output file', dest='output', required=True)
    
    args = parser.parse_args()
    
    lbl, labeled_indexes, tree, result_temp = get_dependency_data(args.lbl, args.guess, args.dep)
    tweets = get_error_analysis(lbl, labeled_indexes, tree, result_temp)
    write_error_analysis_file(args.output, tweets)
        
             
if __name__ == '__main__':
    main(sys.argv[1:])

In [ ]:
with open(labeled_data) as l:
    with open(result_data) as r:
        with open(dep_tree) as d:
            labels = l.readlines()
            results = r.readlines()
            deps = d.readlines()
            tweets = []
            arr = []
            ind = 0
            match = 1
            for t1, t2, t3 in zip(labels, results, deps):
                if t1 in ['\n', '\r\n']:
                    if not match:
                        tweets.append(arr)
                    match = 1
                    arr = []
                else:
                    (word, tag) = t1.strip().split()
                    (w_id, word, pos_tag, head, rel) = t3.strip().split()
                    if tag != t2.strip():
                        match = 0
                    arr.append((w_id, word, pos_tag, head, rel, tag, t2.strip()))